In [1]:
from pathlib import Path
import yaml

In [2]:
config_path = Path("configs/default_missions/search.yaml")

In [3]:
with open(config_path, "r") as f:
    config = yaml.safe_load(f)

In [7]:
config["states"].items()

dict_items([('ROTATE_CLOCK', {'initial': True, 'event': 'object_detection_event', 'expire_time': 10, 'action': {'type': 'rotation', 'args': {'speed': '(0', 0: None, '15)': None, 'angle': 90}}}), ('ROTATE_ANTICLOCK', {'event': 'object_detection_event', 'expire_time': 10, 'action': {'type': 'rotation', 'args': {'speed': '(0', 0: None, '-15)': None, 'angle': 90}}})])

In [ ]:
class Mission(object):
    def __init__(self, node: Node, config_path: str):
        """Mission class for executing a mission from a config file"""
        self.node = node
        with open(config_path, "r") as f:
            self.config = yaml.safe_load(f)

        self.name = self.config["name"]

        self.setup_machine()
        self.register_events()

    def setup_machine(self):
        """Registering the state machine from the config file"""
        self.start_state = f'START_{self.name.upper()}'
        self.finish_state = f'FINISH_{self.name.upper()}'
        self.go_transition = f'go_{self.name.lower()}'
        self.abort_transition = f'abort_{self.name.lower()}' 

        self.machine = AsyncMachine(
            model=self,
            states=[self.start_state, self.finish_state],
            transitions=[[self.abort_transition, "*", self.finish_state]],
            initial=self.start_state,
            auto_transitions=False,
            after_state_change="execute_state",
            before_state_change="leave_state",
        )

        for state, args in self.config['states'].items():
            self.machine.add_state(f'{state}_{self.name.upper()}')
            if 'initial' in args and args['initial']:
                self.machine.add_transition(trigger=self.go_transition, source=self.start_state, dest=state)
        
        for transition in self.config['transitions']:
            self.machine.add_transition(trigger=transition['trigger'], source=transition['source'], dest=transition['dest'].format(start=self.start_state, finish=self.finish_state))

    async def on_enter_FINISH(self):
        """Executing on entering the FINISH state"""
        for event in self.events.values():
            await event.unsubscribe(self.node)
        logger.info("Mission finished")

    async def execute_state(self):
        """Executing as soon as the state is entered"""
        if self.state in self.events:
            await self.events[self.state].subscribe(self.node)
        logger.info(f"{self.state} started")

    async def leave_state(self):
        """Executing before leaving the state"""
        if self.state in self.events:
            await self.events[self.state].unsubscribe(self.node)
        logger.info(f"{self.state} ended")

    def register_events(self):
        """Registering events from the config file"""
        self.events: dict[str, TopicEvent] = {}
        for event in self.config["events"]:
            if event["type"] == "TopicEvent":
                self.events[event["state"]] = TopicEvent(
                    trigger_fn=self.trigger,
                    topic=event["topic"],
                    data=event["data"],
                    trigger=event["trigger"],
                    count=event["count"],
                )
            else:
                raise ValueError("Event type not supported")


In [12]:
for state, args in config['states'].items():
    print(state, args)

SEARCH_CLOCK {'trigger_event': 'object_detection_event', 'expire_time': 10, 'action': 'rotate_clockwise'}
SEARCH_ANTICLOCK {'trigger_event': 'object_detection_event', 'expire_time': 10, 'action': 'rotate_anticlockwise'}


In [14]:
"finish".format(finish="FINISH_SEARCH", start="START_SEARCH")

'finish'